### Homework 2

Send your solutions before 28/10/2021 23:59:59 Moscow time to yanovich.yury@ya.ru. The preferable format of the solution is IPython notebook. Please, set the topic "MMDLS HW2 - <your last name>". For example, "MMDLS HW2 - Yanovich". 

In [1]:
import base58
import binascii
import bitcoin
import hashlib
import os
import ecdsa
from ecdsa import SigningKey, SECP256k1
from prettytable import PrettyTable
from typing import List

<b>Modified helper functions from seminar notebook.</b>

In [2]:
# Function For Hashing: 
# 1. Sha256 
# 2. Ripemd160
def hashing(a):
    first_sha256 = hashlib.sha256(a)
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return_1 = bytes.fromhex('00') + ripemd160.digest()
    return_2 = bytes.fromhex('6f') + ripemd160.digest()
    return return_0, return_1, return_2

# Function for Checksum
def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

# Function to convert to base58    
def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

def compressed_key(d):
    a = d.hex()

    if a[-1] == '0' or a[-1] == '2' or a[-1] == '4' or a[-1] == '6' or a[-1] == '8' or a[-1] == 'a' or a[-1] == 'c' or a[-1] == 'e':
        return bytes.fromhex('02') + d
    else:
        return bytes.fromhex('03') + d

def generate_address(print_output = True):
    # Generating Random number of 32 bytes
    private_key = os.urandom(32).hex()
    private_key_from_hex = bytes.fromhex(private_key)

    # Keeping same private key, Generating Public Key
    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1) 

    # Actual publickey to verify messages that singed using own pvt key
    verification_key = sk.verifying_key
    # Making Public Key:
    public_key = bytes.fromhex('04') + verification_key.to_string()
    compressed_public_key = compressed_key(verification_key.to_string())
    compressed_public_key_hex = compressed_public_key.hex()[:66]
    # Converting public key to address:
    decoded_pubkey, mainnet_pubkey, testnet_pubkey = hashing(public_key)

    # Checksum:
    checksum_main_pubkey = checksum(mainnet_pubkey)
    checksum_test_pubkey = checksum(testnet_pubkey)

    main_address = to_base58(checksum_main_pubkey)
    test_address = to_base58(checksum_test_pubkey)

    if print_output:
        x = PrettyTable()
        x.field_names = ['Description', 'Value']
        x.add_row(['Private Key, hex', private_key])
        x.add_row(['Public Key, hex', public_key.hex()])
        x.add_row(['Compressed Public Key, hex', compressed_public_key_hex])
        x.add_row(['MainNet Address, base58', main_address])
        x.add_row(['Testnet Address, base58', test_address])
        print(x)
        
    return(private_key, public_key.hex(), compressed_public_key_hex, main_address, test_address)

# Generate Transaction
def bytewise_reverse(hex_string: str) -> str:
    i: int = 0
    reversed_string: List[str] = []
    while i != len(hex_string):
        reversed_string.append(hex_string[i] + hex_string[i + 1])
        i = i + 2
    reversed_string.reverse()
    
    return ''.join(reversed_string)

def doublehash256_txid(a: str) -> str: 
    return str(binascii.hexlify(hashlib.sha256(hashlib.sha256(binascii.unhexlify(a)).digest()).digest()), 'ascii')

def public_to_test(script_address: str) -> str:
    decoded_address = base58.b58decode(script_address)
    address_1 = decoded_address[1:-4]
    hex_address = bytes.fromhex('C4') + address_1
    checksum_test_multsignature = checksum(hex_address)
    test_address = to_base58(checksum_test_multsignature)
    
    return test_address

def bitcoin_satoshi_rev_hex(btc):
    return bytewise_reverse(str(hex(int(float(btc) * (10**8)))[2:]).zfill(16))

def bytes_padding(data) -> str:
    return '0' + data if (int(len(data) % 2) != 0) else data

def op_returns() -> str:
    value_return: str = '0'*16
    data: str = input('Enter data to send: ')
    text_from_bytes: str = binascii.b2a_hex(data.encode('utf-8')).decode()
    data_len: List[str] = hex(int(len(text_from_bytes) / 2))[2:]
    partial_data: str = '6a'+ bytes_padding(data_len) + text_from_bytes
        
    return value_return + bytes_padding(hex(int(len(partial_data) / 2))[2:]) + partial_data

def digital_signature(raw_transaction: str) -> str:
    unsigned: str = raw_transaction
    transaction_hash: str = hashlib.sha256(hashlib.sha256(binascii.unhexlify(unsigned)).digest()).digest()
    private_key: str = input('Enter Private Key: ')
    signing_key: str = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)  
    signature: str = str(binascii.hexlify(signing_key.sign_digest(transaction_hash, sigencode=ecdsa.util.sigencode_der_canonize)), 'ascii')
        
    return signature
    
def variable_integer(a) -> str:
    temp = a if type(a) == int else int(len(a) / 2)

    if temp <= int('0xfc', 16):
        return bytes_padding(hex(temp)[2:])
    elif temp > int('0xfc', 16) and temp <= int('0xffff', 16):
        return 'fd' + bytewise_reverse(bytes_padding(hex(temp)[2:]))
    elif temp > int('0xffff', 16) and temp <= int('0xffffffff', 16):
        return 'fe' + bytewise_reverse(bytes_padding(hex(temp)[2:]))
    elif temp > int('0xffffffff', 16) and temp <= int('0xffffffffffffffff', 16):
        return 'ff' + bytewise_reverse(bytes_padding(hex(temp)[2:]))

def transaction_outputs() -> str:
    receiver_address: str = input('Enter receiver\'s address: ')
    value_in_btc: float = float(input('Enter value in BTC to send: '))
    value_in_hex_NYB: str = bitcoin_satoshi_rev_hex(str(value_in_btc))
    receiver_address_decoded: str = base58.b58decode(receiver_address).hex()[2:-8]
    locking_Script: str = ''
    script_length: str = ''
        
    if receiver_address[0] == 'm' or receiver_address[0] == 'n': 
        locking_script = '76' + 'a9' + variable_integer(receiver_address_decoded) + receiver_address_decoded + '88' + 'ac'
        script_length = hex(int(len(locking_script) / 2))[2:]
    elif receiver_address[0] == '2':
        locking_script = 'a9' + variable_integer(receiver_address_decoded) + receiver_address_decoded + '87'
        script_length = variable_integer(locking_script)
            
    return value_in_hex_NYB + script_length + locking_script

def push_data(len_r: str, len_s: str) -> str:
    if len_r == '20' and len_s == '20':
        return str(47)    
    elif len_r == '20' or len_s == '21':
        return str(48)
    elif len_r == '21' or len_s == '20':
        return str(48)
    else:
        return str(49)

def public_key_opcode(a: str) -> str:
    return hex(int(int(len(a)) / 2))[2:]
    
def decode_digital_signature(digital_signature: str, public_key: str) -> str:
    header: str = digital_signature[0:2]
    sig_length: str = digital_signature[2:4]
    r_integer: str = digital_signature[4:6]
    r_length: str = digital_signature[6:8]
    r: str = digital_signature[8:8 + 2 * int(r_length, 16)]
    s_integer: str = digital_signature[8 + 2 * int(r_length, 16): 10 + 2 * int(r_length, 16)]
    s_length: str = digital_signature[10 + 2 * int(r_length, 16): 12 + 2 * int(r_length, 16)]
    s: str = digital_signature[12 + 2 * int(r_length, 16):]
    
    push_data_opcode: str = push_data(r_length, s_length)
    sighash_code: str = '01'
    public_key_push_data_opcode = public_key_opcode(public_key)

    final_sig_script: str = push_data_opcode + header + sig_length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + public_key_push_data_opcode + public_key
    sig_script = public_key_opcode(final_sig_script) + final_sig_script
    
    return sig_script

def place_holder() -> str:
    previous_place_holder: str = input('Enter output script: ')
    script_public_key_previous_operation: str = '76' + 'a9' + hex(int(len(previous_place_holder) / 2))[2:] + previous_place_holder + '88' + 'ac'
    
    return hex(int(len(script_public_key_previous_operation) / 2))[2:] + script_public_key_previous_operation

def tx_inputs_tx_id() -> str:
    previous_tx_hash = input('Enter previous transaction id: ')
    previous_tx_hash_reversed = bytewise_reverse(previous_tx_hash)
    previous_output_index = input('Enter previous output: ')
    previous_output_index = bytewise_reverse('{:08d}'.format(int(previous_output_index)))
    
    return previous_tx_hash_reversed + previous_output_index

def transaction_outputs_v2() -> str:
    receiver_address: str = input('Enter receiver\'s address: ')
    value_in_btc: float = float(input('Enter value in BTC to send: '))
    value_in_hex_NYB: str = bitcoin_satoshi_rev_hex(str(value_in_btc))
    receiver_address_decoded: str = base58.b58decode(receiver_address).hex()[2:-8]
    locking_script: str = ''
        
    if receiver_address[0] == 'm' or receiver_address[0] == 'n': 
        locking_script = '76' + 'a9' + variable_integer(receiver_address_decoded) + receiver_address_decoded + '88' + 'ac'
    elif receiver_address[0] == '2':
        locking_script = 'a9' + variable_integer(receiver_address_decoded) + receiver_address_decoded + '87'
    
    script_length: str = variable_integer(locking_script)
            
    return value_in_hex_NYB + script_length + locking_script

<b>Pre-requisite 1.</b> Generate 3 testnet addresses (address_1, address_2, address_3; should start with either m or n).

In [3]:
generated_addresses = [generate_address(), generate_address(), generate_address()]

+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|        Description         |                                                               Value                                                                |
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------+
|      Private Key, hex      |                                  62bf458d04adc47bf01a2e2ce6c66496cc65ab1c0080b4a89d1c0b561aec2325                                  |
|      Public Key, hex       | 04a8d1b5d62e468ec32011cf143f1d3413ca751266dfc32df90ccd1a6af59d9ce8ee05d884e13e00c778699efd46c95fffe01f8e23de5cf55b08f80977b6611a7f |
| Compressed Public Key, hex |                                 03a8d1b5d62e468ec32011cf143f1d3413ca751266dfc32df90ccd1a6af59d9ce8                                 |
|  MainNet Addre

In [4]:
address_1 = generated_addresses[0][-1]
address_2 = generated_addresses[1][-1]
address_3 = generated_addresses[2][-1]

In [5]:
print(address_1, address_2, address_3, sep='\n')

muTeXE9wNxS1JADkCb1qPZkGcQ8h34t2NN
moMjgk8QBPkNYJ2XS4T5PwrubdJvxQ6hsp
mhLyTJJAxVMvRB7VgR79NVVzbSKNfnpZBt


In [6]:
public_keys = [generated_addresses[0][1], generated_addresses[1][1], generated_addresses[2][1]]
public_keys

['04a8d1b5d62e468ec32011cf143f1d3413ca751266dfc32df90ccd1a6af59d9ce8ee05d884e13e00c778699efd46c95fffe01f8e23de5cf55b08f80977b6611a7f',
 '04373fa53e5ce56abe9ff58900d82b1135c1d54f102a4048a3825f5d664e12ea1762179fe2c70a53dc8aeecf27548f4ddb01fa860c536b90c0d420b3fdce24f4da',
 '04dd24241cbfb790b1c60ceb632b14b270c594d7f70c4d3c3b6a9b955879b652f9ee1a37fbd5b5c6b2e28f65ae7af1d393a8b94feeee44fb8bf2958e7ab8df6da4']

In [7]:
private_keys = [generated_addresses[0][0], generated_addresses[1][0], generated_addresses[2][0]]
private_keys

['62bf458d04adc47bf01a2e2ce6c66496cc65ab1c0080b4a89d1c0b561aec2325',
 '9ea470901eebb2b6ddb5d6c1286376a943fbcb429280b66d7ec50c1e9546f340',
 'ddbc2793a9fdf067ecee11624fc47483e3c157bdf9b174ff22cdb8e317a7b79e']

<b>Pre-requisite 2.</b> From prerequisite 1,  generate 2-3 MultiSignature address (2 e.g. private_keys for {addess_1, address_2, address_3} out of three are required to spend Bitcoins). Note that your 2-3 MultiSignature address will start with 2.

In [8]:
n: int = 2
m: int = 3
public_keys: List[str] = public_keys

raw_script = bitcoin.mk_multisig_script(public_keys, n)
deserialized_script = bitcoin.deserialize_script(raw_script)
script_address = bitcoin.scriptaddr(raw_script)
testnet_address = public_to_test(script_address)

print('MultiSignature Testnet Address: ', testnet_address)
print('Redeem Script:', raw_script)    

MultiSignature Testnet Address:  2MwahZHf6jYmYD8hov5XSsFDGUEMoVt3b2F
Redeem Script: 524104a8d1b5d62e468ec32011cf143f1d3413ca751266dfc32df90ccd1a6af59d9ce8ee05d884e13e00c778699efd46c95fffe01f8e23de5cf55b08f80977b6611a7f4104373fa53e5ce56abe9ff58900d82b1135c1d54f102a4048a3825f5d664e12ea1762179fe2c70a53dc8aeecf27548f4ddb01fa860c536b90c0d420b3fdce24f4da4104dd24241cbfb790b1c60ceb632b14b270c594d7f70c4d3c3b6a9b955879b652f9ee1a37fbd5b5c6b2e28f65ae7af1d393a8b94feeee44fb8bf2958e7ab8df6da453ae


<b>Task 1.</b>(1 point) Calculate double Hash_256 of your name. Take two inputs from the user, i.e., first name and last name. Concatenate string and take SHA256(SHA256(<first name><lastname>)).

In [9]:
def double_hash_256(first: str, second: str) -> str:         
    return str(binascii.hexlify(hashlib.sha256(hashlib.sha256((first + second).encode('utf-8')).digest()).digest()), 'ascii')

In [9]:
first_name: str = input()
last_name: str = input()
    
double_hash: str = double_hash_256(first_name, last_name)

print(f'Double Hash_256 of your name is {str(double_hash)}')

Maria
Manakhova
Double Hash_256 of your name is 4a13d35c36e8a6500bb0958c51c7515a5895a9d89666f66f4660afe7e49d649d


<b>Task 2.</b> (1 point) Get some testnet Bitcoins to your testnet address_1 from any source. (For example: https://coinfaucet.eu/en/btc-testnet/, https://testnet-faucet.mempool.co/, or search "bitcoin testnet faucet") . Provide the Transaction_ID.

In [28]:
# address_1 = muTeXE9wNxS1JADkCb1qPZkGcQ8h34t2NN
# https://blockstream.info/testnet/address/muTeXE9wNxS1JADkCb1qPZkGcQ8h34t2NN
transaction_id: str = '5b4172fe2b7334669c097eaff134e66ae3e102219c0674e6e50ae2564d61197f'
script_public_key_hash: str = '98f1ad52b9245fb7a64cd6922cb2458f2fdf97e7'

<b>Task 3.</b> (2 points) Create a new transaction from address_1 and provide Transaction_ID. Make 2 outputs: 
<ul>
  <li>Output_1 : Some bitcoins to address_2.</li>
  <li>Output_2: Data transfer i.e. First 4 Bytes double Hash_256 of your name. You can send 0 bitcoins in this output.</li>
</ul>

In [10]:
version_number: str = '01000000'
number_inputs: str = '01'
sequence: str = 'ffffffff'

previous_transaction_id: str = input('Enter previous user transaction id: ')
previous_output: int = int(input('Enter previous output (<= 15): '))
previous_public_key_hash: str = input('Enter previous public key hash: ')
    
previous_transaction_id_reversed: str = bytewise_reverse(previous_transaction_id)
output_script: str = '0' + str(previous_output) + '000000'
script: str = '1976a914' + previous_public_key_hash + '88ac'
transaction_ip: str = version_number + number_inputs + previous_transaction_id_reversed + output_script + script + sequence

# Outputs
outputs: int = int(input('Enter number of outputs: '))
    
utxo_outputs = int(input('Enter number of utxo outputs: '))
transaction_output: str = ''
for i in range(utxo_outputs):
    transaction_output = transaction_outputs() + transaction_output
    
data_outputs: int = int(input('Enter number of data outputs: '))
op_return: str = ''
for i in range(data_outputs):
    op_return = op_returns() + op_return
    
out_transaction = '0' + str(outputs) + transaction_output + op_return + '0'*8 + version_number
raw_transaction: str = transaction_ip + out_transaction
    
public_key: str = input('Enter public key (compressed or uncompressed): ')
    
signature: str = digital_signature(raw_transaction)
sig_script = decode_digital_signature(signature, public_key)

print(f'Raw transaction: {raw_transaction}')
print(f'Digital signature: {signature}')
print(f'Sig Script: {sig_script}')

#Broadcasting_Transaction:
broadcasting_transaction: str = version_number + number_inputs + previous_transaction_id_reversed + output_script + sig_script + sequence + out_transaction[:-8]
print(f'Transaction to broadcast: {broadcasting_transaction}')

Enter previous user transaction id: 5b4172fe2b7334669c097eaff134e66ae3e102219c0674e6e50ae2564d61197f
Enter previous output (<= 15): 0
Enter previous public key hash: 98f1ad52b9245fb7a64cd6922cb2458f2fdf97e7
Enter number of outputs: 2
Enter number of utxo outputs: 1
Enter receiver's address: moMjgk8QBPkNYJ2XS4T5PwrubdJvxQ6hsp
Enter value in BTC to send: 0.0005
Enter number of data outputs: 1
Enter data to send: 4a13
Enter public key (compressed or uncompressed): 04a8d1b5d62e468ec32011cf143f1d3413ca751266dfc32df90ccd1a6af59d9ce8ee05d884e13e00c778699efd46c95fffe01f8e23de5cf55b08f80977b6611a7f
Enter Private Key: 62bf458d04adc47bf01a2e2ce6c66496cc65ab1c0080b4a89d1c0b561aec2325
Raw transaction: 01000000017f19614d56e20ae5e674069c2102e1e36ae634f1af7e099c6634732bfe72415b000000001976a91498f1ad52b9245fb7a64cd6922cb2458f2fdf97e788acffffffff0250c30000000000001976a9145602bcc1992be6d20702afcefa6499db212a59d688ac0000000000000000066a04346131330000000001000000
Digital signature: 3044022045807ca6653f7e5c

In [11]:
# address_2 = moMjgk8QBPkNYJ2XS4T5PwrubdJvxQ6hsp
# https://blockstream.info/testnet/address/moMjgk8QBPkNYJ2XS4T5PwrubdJvxQ6hsp
second_transaction_id: str = '02df7e71fbb36ba6c471e46d0d33ded5f38bd762a462c3da592c23b3bd788588'
second_script_public_key_hash: str = '5602bcc1992be6d20702afcefa6499db212a59d6'

<b>Task 4.</b> (4 points) From address_2, Send some bitcoins to your 2-of-3 MultiSignature Address. 

In [12]:
version_number: str = '01000000'
number_inputs: str = '01'
sequence: str = 'ffffffff'

previous_transaction_id: str = input('Enter previous user transaction id: ')
previous_output: int = int(input('Enter previous output (<= 15): '))
previous_public_key_hash: str = input('Enter previous public key hash: ')
    
previous_transaction_id_reversed: str = bytewise_reverse(previous_transaction_id)
output_script: str = '0' + str(previous_output) + '000000'
script: str = '1976a914' + previous_public_key_hash + '88ac'
transaction_ip: str = version_number + number_inputs + previous_transaction_id_reversed + output_script + script + sequence

# Outputs
outputs: int = int(input('Enter number of outputs: '))
    
utxo_outputs = int(input('Enter number of utxo outputs: '))
transaction_output: str = ''
for i in range(utxo_outputs):
    transaction_output = transaction_outputs() + transaction_output
    
data_outputs: int = int(input('Enter number of data outputs: '))
op_return: str = ''
for i in range(data_outputs):
    op_return = op_returns() + op_return
    
out_transaction = '0' + str(outputs) + transaction_output + op_return + '0'*8 + version_number
raw_transaction: str = transaction_ip + out_transaction
    
public_key: str = input('Enter public key (compressed or uncompressed): ')
    
signature: str = digital_signature(raw_transaction)
sig_script = decode_digital_signature(signature, public_key)

print(f'Raw transaction: {raw_transaction}')
print(f'Digital signature: {signature}')
print(f'Sig Script: {sig_script}')

#Broadcasting_Transaction:
broadcasting_transaction: str = version_number + number_inputs + previous_transaction_id_reversed + output_script + sig_script + sequence + out_transaction[:-8]
print(f'Transaction to broadcast: {broadcasting_transaction}')

Enter previous user transaction id: 02df7e71fbb36ba6c471e46d0d33ded5f38bd762a462c3da592c23b3bd788588
Enter previous output (<= 15): 0
Enter previous public key hash: 5602bcc1992be6d20702afcefa6499db212a59d6
Enter number of outputs: 1
Enter number of utxo outputs: 1
Enter receiver's address: 2MwahZHf6jYmYD8hov5XSsFDGUEMoVt3b2F
Enter value in BTC to send: 0.0004
Enter number of data outputs: 0
Enter public key (compressed or uncompressed): 04373fa53e5ce56abe9ff58900d82b1135c1d54f102a4048a3825f5d664e12ea1762179fe2c70a53dc8aeecf27548f4ddb01fa860c536b90c0d420b3fdce24f4da
Enter Private Key: 9ea470901eebb2b6ddb5d6c1286376a943fbcb429280b66d7ec50c1e9546f340
Raw transaction: 0100000001888578bdb3232c59dac362a462d78bf3d5de330d6de471c4a66bb3fb717edf02000000001976a9145602bcc1992be6d20702afcefa6499db212a59d688acffffffff01409c00000000000017a9142f8f6b7d505fec6ad7fc4a3a12ea1fbe8c52a3df870000000001000000
Digital signature: 304402205ad39e6151e5cd4638674f7830b4fa9220040f6d64bb8b393d991813690cc911022032d5a2

In [13]:
# multisignature address = 2MwahZHf6jYmYD8hov5XSsFDGUEMoVt3b2F
# https://blockstream.info/testnet/address/2MwahZHf6jYmYD8hov5XSsFDGUEMoVt3b2F
third_transaction_id: str = '9c947d1abeace962a4ef32db0411be0c589eb8c944dcad65a9177ac243c5483f'

<b>Task 5.</b> (2 points) From 2-of -3 MultiSignature Address. Send some bitcoins back to address_1. 

In [15]:
version: str = '01000000'
lock_time: str = '00000000'
sig_hash_code: str = '01000000'
    
signatures_required: int = int(input('Enter number of signatures required: '))
input_multi_signature: str = '01' + tx_inputs_tx_id()

redeem_script: str = input('Enter redeem script: ')
redeem_script = variable_integer(redeem_script) + redeem_script

transaction_input_value: str = input_multi_signature + redeem_script + 'f'*8

utxo_outputs: int = int(input('Enter number of outputs: '))
transaction_output: str = ''
for i in range(utxo_outputs):
    transaction_output = transaction_outputs_v2() + transaction_output
    
next_output_data: str = variable_integer(utxo_outputs) + transaction_output
transaction_to_sign: str = version + transaction_input_value + next_output_data + lock_time + sig_hash_code

signatures: str = ''
for i in range(signatures_required):
    digital_signature_: str = digital_signature(transaction_to_sign) + '01'
    signatures = signatures + (variable_integer(digital_signature_) + digital_signature_)
    
sig_script_places: str = '00' + signatures + '4c' + redeem_script 
sig_script: str = variable_integer(sig_script_places) + sig_script_places

broadcasting_transaction = version + input_multi_signature + sig_script + 'f'*8 + next_output_data + lock_time
print('Transaction to broadcast: ', broadcasting_transaction)

Enter number of signatures required: 2
Enter previous transaction id: 9c947d1abeace962a4ef32db0411be0c589eb8c944dcad65a9177ac243c5483f
Enter previous output: 0
Enter redeem script: 524104a8d1b5d62e468ec32011cf143f1d3413ca751266dfc32df90ccd1a6af59d9ce8ee05d884e13e00c778699efd46c95fffe01f8e23de5cf55b08f80977b6611a7f4104373fa53e5ce56abe9ff58900d82b1135c1d54f102a4048a3825f5d664e12ea1762179fe2c70a53dc8aeecf27548f4ddb01fa860c536b90c0d420b3fdce24f4da4104dd24241cbfb790b1c60ceb632b14b270c594d7f70c4d3c3b6a9b955879b652f9ee1a37fbd5b5c6b2e28f65ae7af1d393a8b94feeee44fb8bf2958e7ab8df6da453ae
Enter number of outputs: 1
Enter receiver's address: muTeXE9wNxS1JADkCb1qPZkGcQ8h34t2NN
Enter value in BTC to send: 0.0002
Enter Private Key: 62bf458d04adc47bf01a2e2ce6c66496cc65ab1c0080b4a89d1c0b561aec2325
Enter Private Key: ddbc2793a9fdf067ecee11624fc47483e3c157bdf9b174ff22cdb8e317a7b79e
Transaction to broadcast:  01000000013f48c543c27a17a965addc44c9b89e580cbe1104db32efa462e9acbe1a7d949c00000000fd5e010048304502

In [16]:
final_transaction_id: str = '28343ca90629b9e3acbd56134ae84d28effcda384f25f73e0a7e109bb9932322'